# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wairoa,-39.0333,177.3667,291.23,99,100,3.13,NZ,1674088629
1,1,georgetown,5.4112,100.3354,298.12,87,20,2.06,MY,1674088507
2,2,butaritari,3.0707,172.7902,300.30,78,89,7.09,KI,1674088631
3,3,cabo san lucas,22.8909,-109.9124,295.32,64,20,4.63,MX,1674088631
4,4,bredasdorp,-34.5322,20.0403,293.15,84,6,2.61,ZA,1674088632


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# 21 degree celsius=294.15 & 27 degree celsius=300.15

city_temp=city_data_df.loc[(city_data_df['Max Temp'] >=294.15) & (city_data_df['Max Temp'] <300.15)]

city_wind=city_temp.loc[city_temp['Wind Speed']<4.5]

city_zero_cloudiness=city_wind.loc[city_wind['Cloudiness']==0]

# Drop any rows with null values
city_final_data=city_zero_cloudiness.dropna()

# Display sample data
city_final_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,coihaique,-45.5752,-72.0662,298.49,40,0,0.00,CL,1674088641
59,59,salalah,17.0151,54.0924,296.20,68,0,1.03,OM,1674088681
128,128,port elizabeth,-33.9180,25.5701,297.77,81,0,4.12,ZA,1674088746
131,131,nhulunbuy,-12.2333,136.7667,299.49,83,0,3.09,AU,1674088749
285,285,brownsville,25.9018,-97.4975,299.31,87,0,4.12,US,1674088887
352,352,bahia blanca,-38.7196,-62.2724,294.44,92,0,3.60,AR,1674088776
384,384,maputo,-25.9653,32.5892,297.97,83,0,1.03,MZ,1674088976
485,485,illapel,-31.6308,-71.1653,294.16,66,0,1.56,CL,1674089073
495,495,natal,-5.7950,-35.2094,296.51,100,0,1.54,BR,1674089082
496,496,alice town,25.7167,-79.3000,296.32,66,0,3.97,BS,1674089082


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_final_data[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df.insert(5,"Hotel Name", "")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,coihaique,CL,-45.5752,-72.0662,40,
59,salalah,OM,17.0151,54.0924,68,
128,port elizabeth,ZA,-33.9180,25.5701,81,
131,nhulunbuy,AU,-12.2333,136.7667,83,
285,brownsville,US,25.9018,-97.4975,87,
352,bahia blanca,AR,-38.7196,-62.2724,92,
384,maputo,MZ,-25.9653,32.5892,83,
485,illapel,CL,-31.6308,-71.1653,66,
495,natal,BR,-5.7950,-35.2094,100,
496,alice town,BS,25.7167,-79.3000,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city= row["City"]
    latitude= row["Lat"]
    longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

   
hotel_df

Starting hotel search
coihaique - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
port elizabeth - nearest hotel: Waterford Hotel
nhulunbuy - nearest hotel: No hotel found
brownsville - nearest hotel: Cameron Hotel
bahia blanca - nearest hotel: Hotel Austral Bahía
maputo - nearest hotel: Mozaika
illapel - nearest hotel: Domingo Ortiz de Rosas
natal - nearest hotel: Natal Palace Hotel
alice town - nearest hotel: No hotel found
marataizes - nearest hotel: Saveiros Palace Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
13,coihaique,CL,-45.5752,-72.0662,40,No hotel found
59,salalah,OM,17.0151,54.0924,68,Muscat International Hotel
128,port elizabeth,ZA,-33.9180,25.5701,81,Waterford Hotel
131,nhulunbuy,AU,-12.2333,136.7667,83,No hotel found
285,brownsville,US,25.9018,-97.4975,87,Cameron Hotel
352,bahia blanca,AR,-38.7196,-62.2724,92,Hotel Austral Bahía
384,maputo,MZ,-25.9653,32.5892,83,Mozaika
485,illapel,CL,-31.6308,-71.1653,66,Domingo Ortiz de Rosas
495,natal,BR,-5.7950,-35.2094,100,Natal Palace Hotel
496,alice town,BS,25.7167,-79.3000,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols=["Hotel Name","Country"]
)


# Display the map
map_plot_2 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)